In [57]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Essential for Jupyter notebooks. It keeps the plots visible directly below the cells
# rather than popping up in a separate window or not showing at all.
%matplotlib inline 

# Defining relative paths here rather than absolute ones. 
RAW_PATH = '../data/bangalore_rentals_expanded_5000_v1.csv'
CLEAN_PATH = '../data/cleaned_data.csv'

# Adding a sanity check before trying to load data. 
if not os.path.exists(RAW_PATH):
    print(f"Error: File not found at {RAW_PATH}")
    print(" Make sure the file name matches exactly and is in the 'data' folder.")

else:
    print(f"Found data at: {RAW_PATH}")
    print(" Ready to load!")

Found data at: ../data/bangalore_rentals_expanded_5000_v1.csv
 Ready to load!


In [63]:
# 2. Noise Reduction: Remove ID columns
# listing_id and society are unique strings that don't help a model "learn" price rules.
# We drop them to make the model "lighter."
df_cleaned = df.drop(columns=['listing_id', 'society'])
display(df_cleaned.head(10))

,area_type,availability,location,zone,market_status,property_type,size_bhk,total_sqft,bath,balcony,...,has_refrigerator,has_washing_machine,water_source,nearest_metro_station,dist_to_metro_km,legal_security_deposit,market_security_deposit,nearby_hospitals,commute_time_peak_mins,is_ready
0,Super built-up Area,Ready To Move,Mahadevapura,East,Normal,Independent House,3,1571,3,1,...,0,0,Tanker,Mahadevapura,0.7,111906,503577,4,51,1
1,Super built-up Area,Ready To Move,Nagarbhavi,West,Normal,Independent House,2,1114,2,3,...,1,1,Borewell,Vijayanagar,3.5,90990,454950,3,52,1
2,Super built-up Area,Within 30 Days,Banashankari,South,Normal,Apartment,3,1597,3,1,...,0,0,Cauvery,Banashankari,0.4,121498,485992,6,46,0
3,Super built-up Area,Ready To Move,Banaswadi,East,Normal,Apartment,2,1300,2,3,...,1,1,Tanker,NaN,10.0,107024,428096,5,47,1
4,Built-up Area,Ready To Move,Marathahalli,East,Hot,Independent House,3,1588,4,1,...,0,0,Borewell,Kundalahalli,3.0,152132,532462,8,29,1
5,Super built-up Area,Ready To Move,Electronic City,South,Normal,Builder Floor,2,1010,2,0,...,1,1,Cauvery,Electronic City,0.8,35812,143248,4,65,1
6,Built-up Area,Ready To Move,Hoskote,East,Cold,Builder Floor,2,1273,3,3,...,1,1,Tanker,NaN,10.0,65954,296793,0,89,1
7,Super built-up Area,Within 30 Days,Vidyaranyapura,North,Normal,Builder Floor,3,1749,3,1,...,0,0,Tanker,NaN,10.0,125318,563931,7,52,0
8,Built-up Area,Within 30 Days,Electronic City,South,Normal,Apartment,2,1167,2,0,...,0,0,Cauvery + Borewell,Electronic City,0.8,35288,105864,5,45,0
9,Built-up Area,Ready To Move,Richmond Town,Central,Hot,Builder Floor,2,1347,3,0,...,1,1,Borewell,MG Road,1.5,185178,925890,10,36,1


In [59]:
# 3. Handling Outliers (The "Realism" Filter)
# Extreme values (like a 10,000 sqft house) can skew the average rent.
# We 'Clip' these values using the IQR (Interquartile Range) method.

def clip_outliers(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
    data[column] = data[column].clip(lower=lower, upper=upper)
    return data

# there are 2 columns wich has outliers and those should be handled correctly
df_cleaned = clip_outliers(df_cleaned, 'rent_price_inr_per_month')
df_cleaned = clip_outliers(df_cleaned, 'total_sqft')

In [60]:
# 4. Handling Missing Metro Data
# Some areas are far from the metro. Instead of 'NaN', we mark them as 'No Metro'.
df_cleaned['nearest_metro_station'] = df_cleaned['nearest_metro_station'].fillna('No Metro')

## Text Standerdization

In [61]:
import re

def robust_standardize(text):
    if not isinstance(text, str):
        return text
    # 1. Lowercase & Strip spaces
    text = text.lower().strip()
    # 2. Replace hyphens/slashes with a space to unify "Semi-Furnished" and "Semi Furnished"
    text = re.sub(r'[-/]', ' ', text)
    # 3. Remove double spaces
    text = ' '.join(text.split())
    return text

# Apply to all text columns
text_cols = df_cleaned.select_dtypes(include=['object']).columns
for col in text_cols:
    df_cleaned[col] = df_cleaned[col].apply(robust_standardize)

/var/folders/qd/_69mz3nj7dvfh42ryyzvy5nc0000gn/T/ipykernel_8924/3377910253.py:15: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  text_cols = df_cleaned.select_dtypes(include=['object']).columns


In [64]:
# Map common variations to a single standard
mapping = {
    'semi furnished': 'semi-furnished',
    'partially furnished': 'semi-furnished',
    'fully furnished': 'furnished',
    'unfurnished': 'un-furnished'
}

# Apply to a specific column (e.g., 'furnishing_status')
df_cleaned['furnishing'] = df_cleaned['furnishing'].replace(mapping)

In [65]:
# Print unique values for all text columns to check for leftovers
for col in text_cols:
    print(f"Unique values in {col}: {df_cleaned[col].unique()}")

Unique values in area_type: <StringArray>
['Super built-up Area', 'Built-up Area']
Length: 2, dtype: str
Unique values in availability: <StringArray>
['Ready To Move', 'Within 30 Days']
Length: 2, dtype: str
Unique values in location: <StringArray>
[                 'Mahadevapura',                    'Nagarbhavi',
                  'Banashankari',                     'Banaswadi',
                  'Marathahalli',               'Electronic City',
                       'Hoskote',                'Vidyaranyapura',
                 'Richmond Town',                        'Anekal',
              'Old Airport Road',                       'Varthur',
                    'BTM Layout',                      'KR Puram',
                       'MG Road',                         'Hoodi',
                   'Koramangala',                        'Domlur',
                  'Kalyan Nagar',          'Rajarajeshwari Nagar',
                 'Doddaballapur',                   'Devanahalli',
              

In [66]:
# 1. Identify text columns
text_cols = df_cleaned.select_dtypes(include=['object']).columns

for col in text_cols:
    # Strip spaces, unify hyphens/slashes to spaces, then Title Case
    df_cleaned[col] = (df_cleaned[col]
                       .str.strip()
                       .str.replace(r'[-/]', ' ', regex=True)
                       .str.replace(r'\s+', ' ', regex=True) # Removes double spaces
                       .str.title())

# 2. Check how many unique categories you have now
for col in text_cols:
    print(f"{col}: {df_cleaned[col].nunique()} unique values")

area_type: 2 unique values
availability: 2 unique values
location: 48 unique values
zone: 5 unique values
market_status: 3 unique values
property_type: 3 unique values
furnishing: 3 unique values
parking: 3 unique values
facing: 6 unique values
building_age: 4 unique values
water_source: 5 unique values
nearest_metro_station: 23 unique values


/var/folders/qd/_69mz3nj7dvfh42ryyzvy5nc0000gn/T/ipykernel_8924/3513867226.py:2: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  text_cols = df_cleaned.select_dtypes(include=['object']).columns


## Ordered_data = label encoding

In [69]:
# Define the order for each column
ordinal_mappings = {
    'furnishing': {'Unfurnished': 0, 'Semi Furnished': 1, 'Fully Furnished': 2},
    'market_status': {'Cold': 0, 'Warm': 1, 'Hot': 2},
    'building_age': {'10+ Years': 0, '5-10 Years': 1, '1-5 Years': 2, 'New': 3}
}

# Apply the mapping
for col, mapping in ordinal_mappings.items():
    df_cleaned[col] = df_cleaned[col].map(mapping)

## binary encoding for parking

In [71]:
print(df_cleaned.columns.tolist())

['area_type', 'availability', 'location', 'zone', 'market_status', 'property_type', 'size_bhk', 'total_sqft', 'bath', 'balcony', 'furnishing', 'parking', 'facing', 'building_age', 'distance_to_major_office_km', 'nearby_office_seats_3km', 'nearby_malls_5km', 'nearby_schools_3km', 'mid_income_share', 'higher_income_share', 'buy_probability', 'rent_price_inr_per_month', 'gym_nearby', 'park_nearby', 'swimming_pool', 'food_delivery', 'has_ac', 'has_refrigerator', 'has_washing_machine', 'water_source', 'nearest_metro_station', 'dist_to_metro_km', 'legal_security_deposit', 'market_security_deposit', 'nearby_hospitals', 'commute_time_peak_mins', 'is_ready']


In [72]:
print(df_cleaned['parking'].unique())

<StringArray>
['1 Car', nan, 'Covered', '2 Car']
Length: 4, dtype: str


In [73]:
# If it contains values like 'Two Wheeler', 'Four Wheeler', 'Both', or 'No'
# You can create binary columns based on the text found
df_cleaned['has_2_wheeler_parking'] = df_cleaned['parking'].str.contains('2|Two', case=False).astype(int)
df_cleaned['has_4_wheeler_parking'] = df_cleaned['parking'].str.contains('4|Four', case=False).astype(int)

# Then drop the original text column
df_cleaned.drop('parking', axis=1, inplace=True)

## Date-Time conversion

In [74]:
from datetime import datetime

# 1. Convert availability to datetime objects
# errors='coerce' handles "Ready to Move" text by making it NaT (Not a Time)
df_cleaned['availability_date'] = pd.to_datetime(df_cleaned['availability'], errors='coerce')

def categorize_availability(date):
    if pd.isnull(date): 
        return 'Ready to Move'
    
    days_diff = (date - datetime.now()).days
    
    if days_diff <= 0:
        return 'Ready to Move'
    elif days_diff <= 15:
        return 'Ready to Move' # As per your logic "within 15 ready"
    elif days_diff <= 30:
        return 'Within 30 Days'
    else:
        return 'Long Term'

# Apply the logic
df_cleaned['availability_status'] = df_cleaned['availability_date'].apply(categorize_availability)

# Now Label Encode this new status since it is ordered
availability_map = {'Ready to Move': 2, 'Within 30 Days': 1, 'Long Term': 0}
df_cleaned['availability_score'] = df_cleaned['availability_status'].map(availability_map)

/var/folders/qd/_69mz3nj7dvfh42ryyzvy5nc0000gn/T/ipykernel_8924/1846722829.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_cleaned['availability_date'] = pd.to_datetime(df_cleaned['availability'], errors='coerce')


In [75]:
# Create a binary column: 1 if a station name exists, 0 if it's 'No' or empty
df_cleaned['has_metro'] = df_cleaned['nearest_metro_station'].apply(lambda x: 0 if str(x).lower() == 'no' or pd.isna(x) else 1)

# Keep the station name column separate for your LLM suggestion feature
# (Do not encode the actual names if you want to use them as text output)

## One-hot encoding for unordered text columns

In [94]:
# Check if it's hidden or misspelled
print([col for col in df_cleaned.columns if 'diet' in col.lower()])

# If you find it, add it to your encoding list
unordered_cols = ['area_type', 'property_type', 'facing', 'water_source', 'Zone', 'dietary_preference']
# Note: This will only work if 'dietary_preference' is in the list above!

['dietary_preference_Veg Only']


## handleing location

In [96]:
# 1. Weights
zone_weight = {'South': 3, 'East': 3, 'Central': 3, 'North': 2, 'West': 2}
status_weight = {'Hot': 3, 'Warm': 2, 'Cold': 1}

# 2. Function with consistent lowercase 'zone'
def calculate_location_rank(row):
    # Use .get to safely access the column values
    z_val = row.get('zone', 'North') 
    s_val = row.get('market_status', 'Cold')
    
    z_score = zone_weight.get(z_val, 1)
    s_score = status_weight.get(s_val, 1)
    return (z_score * 10) + s_score

# 3. Create the index
df_cleaned['location_premium_index'] = df_cleaned.apply(calculate_location_rank, axis=1)

# 4. Final Encoding - ensure 'zone' is lowercase here too
# Add dietary_preference and others to the same call to be efficient
target_unordered = ['area_type', 'property_type', 'facing', 'water_source', 'dietary_preference', 'zone']

# Only encode columns that actually exist in the dataframe right now
existing_to_encode = [c for c in target_unordered if c in df_cleaned.columns]

df_final = pd.get_dummies(df_cleaned, columns=existing_to_encode, drop_first=True)

print("Final encoded columns:", df_final.columns.tolist())

Final encoded columns: ['pets_allowed', 'dietary_preference_Veg Only', 'location_premium_index']


## Final check list of all the features and making sure the cleaned data was generated 

In [100]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

# 1. Configuration & Data Loading
RAW_PATH = '../data/bangalore_rentals_expanded_5000_v1.csv'
df = pd.read_csv(RAW_PATH)

# 2. Initial Noise Reduction
# Dropping ID-like columns that don't help the model learn
df_cleaned = df.drop(columns=['listing_id', 'society'], errors='ignore')

# 3. Handling Outliers (Clipping)
# Restricting extreme rent and sqft values using the IQR method
def clip_outliers(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
    data[column] = data[column].clip(lower=lower, upper=upper)
    return data

df_cleaned = clip_outliers(df_cleaned, 'rent_price_inr_per_month')
df_cleaned = clip_outliers(df_cleaned, 'total_sqft')

# 4. Text Standardization & Cleaning
# Unifying strings like "Semi-Furnished" and "Semi Furnished" to "Semi Furnished"
def robust_standardize(text):
    if not isinstance(text, str): return text
    text = re.sub(r'[-/]', ' ', text.lower().strip())
    return ' '.join(text.split()).title()

text_cols = df_cleaned.select_dtypes(include=['object']).columns
for col in text_cols:
    df_cleaned[col] = df_cleaned[col].apply(robust_standardize)

# Filling missing metro data before binary conversion
df_cleaned['nearest_metro_station'] = df_cleaned['nearest_metro_station'].fillna('No Metro')

# 5. Feature Engineering: Location Premium Index
# We calculate this using raw 'Zone' and 'market_status' BEFORE encoding them
zone_weight = {'South': 3, 'East': 3, 'Central': 3, 'North': 2, 'West': 2}
status_weight = {'Hot': 3, 'Warm': 2, 'Cold': 1}

def calculate_location_rank(row):
    z_val = row.get('Zone', row.get('zone', 'North')) 
    s_val = row.get('market_status', 'Cold')
    return (zone_weight.get(z_val, 1) * 10) + status_weight.get(s_val, 1)

df_cleaned['location_premium_index'] = df_cleaned.apply(calculate_location_rank, axis=1)

# 6. Ordinal & Binary Encodings
# Mapping furnishing and building age to ranked integers
ordinal_map = {
    'furnishing': {'Unfurnished': 0, 'Semi Furnished': 1, 'Fully Furnished': 2},
    'building_age': {'10+ Years': 0, '5-10 Years': 1, '1-5 Years': 2, 'New': 3}
}
for col, mapping in ordinal_map.items():
    if col in df_cleaned.columns:
        df_cleaned[col] = df_cleaned[col].map(mapping).fillna(0).astype(int)

# Availability Logic: Converting dates to a score (2=Ready, 1=Soon, 0=Long term)
df_cleaned['availability_date'] = pd.to_datetime(df_cleaned['availability'], errors='coerce')
df_cleaned['availability_score'] = df_cleaned['availability_date'].apply(
    lambda x: 2 if pd.isnull(x) or (x - datetime.now()).days <= 15 else (1 if (x - datetime.now()).days <= 30 else 0)
)

# Metro and Parking Binary features
df_cleaned['has_metro'] = df_cleaned['nearest_metro_station'].apply(lambda x: 0 if str(x).lower() in ['no', 'nan', 'no metro'] else 1)
df_cleaned['has_2_wheeler_parking'] = df_cleaned['two_wheeler_parking'].astype(str).str.contains('2|Two', case=False).astype(int)
df_cleaned['has_4_wheeler_parking'] = df_cleaned['four_wheeler_parking'].astype(str).str.contains('4|Four', case=False).astype(int)

# 7. One-Hot Encoding (Unordered Columns)
# Includes your new 'dietary_preference' column
unordered_cols = ['area_type', 'property_type', 'facing', 'water_source', 'dietary_preference', 'zone']
to_encode = [c for c in unordered_cols if c in df_cleaned.columns]
df_final = pd.get_dummies(df_cleaned, columns=to_encode, drop_first=True)

# 8. Final Dataset Cleanup
# Dropping raw text and redundant columns as their value is now in numeric features
cols_to_drop = [
    'location', 'nearest_metro_station', 'availability', 'market_status', 
    'parking', 'availability_date', 'pets_allowed'
]
df_final.drop(columns=[c for c in cols_to_drop if c in df_final.columns], inplace=True)

# 9. Export Cleaned Data
df_final.to_csv('../data/cleaned_data.csv', index=False)

print(f"Final cleaned dataset shape: {df_final.shape}")
print("All encodings merged. Data saved to '../data/cleaned_data.csv'")

Final cleaned dataset shape: (5000, 45)
All encodings merged. Data saved to '../data/cleaned_data.csv'


/var/folders/qd/_69mz3nj7dvfh42ryyzvy5nc0000gn/T/ipykernel_8924/4220137101.py:34: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  text_cols = df_cleaned.select_dtypes(include=['object']).columns
/var/folders/qd/_69mz3nj7dvfh42ryyzvy5nc0000gn/T/ipykernel_8924/4220137101.py:64: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_cleaned['availability_date'] = pd.to_datetime(df_cleaned['availability'], errors='coerce')


In [101]:
# 1. Define the 'Leaked' columns that provide the answer too easily
leakage_cols = ['legal_security_deposit', 'market_security_deposit']

# 2. Create the Final Dataframe by dropping them
# We keep the target 'rent_price_inr_per_month' in the CSV, but remove it for X
df_final_no_leakage = df_final.drop(columns=leakage_cols)

# 3. Save this as your "Gold Standard" dataset
output_file = '../data/cleaned_data_v2_no_leakage.csv'
df_final_no_leakage.to_csv(output_file, index=False)

print(f"Success! Your final dataset now has {df_final_no_leakage.shape[1]} columns.")
print(f"File saved to: {output_file}")

# 4. Verify the new training shape
X = df_final_no_leakage.drop(columns=['rent_price_inr_per_month'])
print(f"Machine Learning model will now train on {X.shape[1]} legitimate features.")

Success! Your final dataset now has 43 columns.
File saved to: ../data/cleaned_data_v2_no_leakage.csv
Machine Learning model will now train on 42 legitimate features.
